In [1]:
#Импорт библиотек
import os
import mat_properties as prop
import numpy as n
import pandas as pd
import cotel_old as cotel
import nasos
import time
from scipy.optimize import root

#Время начала расчёта
start_time = time.time()

#Основные константы
calctolerance=10**-3
calcmethod = "hybr"
KPDnasos = 0.8074
KPD=0.99

#таблица номинального режима
gas_streams0   = pd.read_excel('streams0.xlsx', sheet_name='gas',  index_col=0)
water_streams0 = pd.read_excel('streams0.xlsx', sheet_name='water',index_col=0)
#таблица задаваемых значений (заменится на интерфейс)
gas_streamsZAD    = pd.read_excel('streams.xlsx',  sheet_name='gas',  index_col=0)
water_streamsZAD  = pd.read_excel('streams.xlsx',  sheet_name='water',index_col=0)
#рабочая таблица (=номинал в 1 итерации)
gas_streams = gas_streams0.copy()
water_streams = water_streams0.copy()

#чтение с вводного экселя входных данных
Tgas = gas_streamsZAD.loc['GTU-PEVD','T']
Pgas = gas_streamsZAD.loc['GTU-PEVD','P']
Ggas = gas_streamsZAD.loc['GTU-PEVD','G']
Propgas = gas_streamsZAD.loc['GTU-PEVD','N2':'Ar']
Pvd = water_streamsZAD.loc['IVD-PEVD','P']
Pnd = water_streamsZAD.loc['IND-PPND','P']
Twat0 = water_streamsZAD.loc['X-GPK','T']

#запись состава газов в раб табл
gas_streams['N2'] = Propgas[0]
gas_streams['O2'] = Propgas[1]
gas_streams['CO2'] = Propgas[2]
gas_streams['H2O'] = Propgas[3]
gas_streams['Ar'] = Propgas[4]

#Состав газов в номинальном режиме и в частичной нагрузке
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
fractiongas0 = (gas_streams0.at['GTU-PEVD','N2'],gas_streams0.at['GTU-PEVD','O2'],gas_streams0.at['GTU-PEVD','CO2'],gas_streams0.at['GTU-PEVD','H2O'],gas_streams0.at['GTU-PEVD','Ar'])
fractiongas  = (gas_streams.at['GTU-PEVD','N2'],gas_streams.at['GTU-PEVD','O2'],gas_streams.at['GTU-PEVD','CO2'],gas_streams.at['GTU-PEVD','H2O'],gas_streams.at['GTU-PEVD','Ar'])

#Считывание рефпропа
RP = prop.init_REFPROP(r'C:\Program Files (x86)\REFPROP')
os.environ['RPPREFIX'] = r'C:/Program Files (x86)/REFPROP'
gas1 = prop.Materials_prop(gasmix,fractiongas, prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)
gas0 = prop.Materials_prop(gasmix,fractiongas0, prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)
water=prop.Materials_prop('water',[1.0,0,0,0,0], prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)

#Задание в раб табл параметров газа (1 строка)
gas_streams.loc['GTU-PEVD','T'] = Tgas
gas_streams['G'] = Ggas
gas_streams['P'] = Pgas
gas_streams.loc['GTU-PEVD','H'] = gas1.p_t(Pgas,Tgas)['h']

#Задание в раб табл параметров по воде
water_streams.loc['IVD-PEVD':'PEN-EVD','P'] = Pvd
water_streams.loc['IND-PPND':'X-GPK','P'] = Pnd
water_streams.loc['BND-PEN','P'] = Pnd
water_streams.loc['X-GPK','T'] = Twat0

#Расчёт параметров на насыщении в испарительных поверхностях (в 1 итерации приближение: недогрев до н.п. на входе 3°С)
water_streams.loc['IVD-PEVD','T'] = water.p_q(Pvd,1)['T']
water_streams.loc['IVD-PEVD','H'] = water.p_q(Pvd,1)['h']
water_streams.loc['IND-PPND','T'] = water.p_q(Pnd,1)['T']
water_streams.loc['IND-PPND','H'] = water.p_q(Pnd,1)['h']
water_streams.loc['EVD-IVD','T'] = water.p_q(Pvd,1)['T']-3
water_streams.loc['GPK-IND','T'] = water.p_q(Pnd,1)['T']-3
water_streams.loc['EVD-IVD','H'] = water.p_t(Pvd,water_streams.loc['EVD-IVD','T'])['h']
water_streams.loc['GPK-IND','H'] = water.p_t(Pnd,water_streams.loc['GPK-IND','T'])['h']

#Расчёт
it=60
for i in range(it):
    ######################################################################
    #Связвка высокого давления
    for j in range(it):
        
        #Связка ПЕВД+ИВД
        for k in range(it):
            
            #Расчёт ПЕВД
            PEVD = cotel.heatex('GTU-PEVD','PEVD-IVD','IVD-PEVD','PEVD-DROS',       
                                KPD,calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
            gas_streams.loc['PEVD-IVD','T':'G'] = [PEVD[0],PEVD[1],PEVD[2],PEVD[3]]
            water_streams.loc['PEVD-DROS','T':'G'] = [PEVD[4],PEVD[5],PEVD[6],PEVD[7]]
            
            #Расчёт ИВД
            IVD  = cotel.vapor ('PEVD-IVD','IVD-EVD','EVD-IVD','IVD-PEVD',           KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
            gas_streams.loc['IVD-EVD','T':'G'] = [IVD[0],IVD[1],IVD[2],IVD[3]]
            water_streams.loc['IVD-PEVD','T':'G'] = [IVD[4],IVD[5],IVD[6],IVD[7]]
            
            #Переопределение расхода в ВД
            water_streams.loc['PEVD-DROS':'PEN-EVD','G'] = IVD[7]
            water_streams.loc['BND-PEN','G'] = IVD[7]
            
            #Баланс ПЕВД+ИВД
            Qgas = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['PEVD-IVD','H'])
            Qwat = water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['IVD-PEVD','H'])
            print('dQ/Q ПЕВД+ИВД',((Qgas-Qwat)/Qgas*100))
            if  abs((Qgas-Qwat)/Qgas*100) < calctolerance:
                break;
        
        #Расчёт ЭВД
        EVD = cotel.heatex('IVD-EVD','EVD-PPND','PEN-EVD','EVD-IVD',               KPD,calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
        gas_streams.loc['EVD-PPND','T':'G'] = [EVD[0],EVD[1],EVD[2],EVD[3]]
        water_streams.loc['EVD-IVD','T':'G'] = [EVD[4],EVD[5],EVD[6],EVD[7]]
        
        #Баланс ПЕВ+ИВД+ЭВД
        Qgas1 = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['IVD-EVD','H'])
        Qwat1 = water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['EVD-IVD','H'])
        print('dQ/Q ПЕВД+ИВД+ЭВД',((Qgas1-Qwat1)/Qgas1*100))
        if  abs((Qgas1-Qwat1)/Qgas1*100) < calctolerance:
            break;
    print("ВД: --- %s сек. ---" % round((time.time() - start_time),1))
    ######################################################################
    
    #Для сходимости
    if i==0:
        gas_streams.loc['PPND-IND','T'] = gas_streams.loc['EVD-PPND','T'] -15
        gas_streams.loc['PPND-IND','H'] = gas1.p_t(gas_streams.loc['PPND-IND','P'],gas_streams.loc['PPND-IND','T'])['h']
    
    ######################################################################
    #Связка низкого давления
    for j in range(it):
        
        #Связка ППНД+ИНД
        for k in range(it):
            
            #Расчёт ППНД
            PPND = cotel.heatex('EVD-PPND','PPND-IND','IND-PPND','PPND-TURB',    KPD,calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
            gas_streams.loc['PPND-IND','T':'G']    = [PPND[0],PPND[1],PPND[2],PPND[3]]
            water_streams.loc['PPND-TURB','T':'G'] = [PPND[4],PPND[5],PPND[6],PPND[7]]
            
            #Расчёт ИНД
            IND  = cotel.vaporND('PPND-IND','IND-GPK','GPK-IND','IND-PPND',      KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
            gas_streams.loc['IND-GPK','T':'G']    = [IND[0],IND[1],IND[2],IND[3]]
            water_streams.loc['IND-PPND','T':'G'] = [IND[4],IND[5],IND[6],IND[7]]
            
            #Переопределение расхода в НД
            water_streams.loc['PPND-TURB':'IND-PPND','G'] = IND[7]
            
            #ПЭН
            water_streams.loc['BND-PEN','T':'G'] = [IND[9],IND[10],IND[11],IND[12]]
            PEN=nasos.nasos('BND-PEN','PEN-EVD',water,KPDnasos,water_streams).calc()
            water_streams.loc['PEN-EVD','T':'G'] = [PEN[0],PEN[1],PEN[2],PEN[3]]
            
            #Баланс ППНД+ИНД
            Qgas = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['IND-GPK','H'])
            Qwat = water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['IND-PPND','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
            print('dQ/Q ППНД+ИНД',((Qgas-Qwat)/Qgas*100))
            if  abs((Qgas-Qwat)/Qgas*100) < calctolerance:
                break;

        #Расчёт расхода в ГПК
        water_streams.loc['X-GPK','G'] = water_streams.at['PPND-TURB','G']+water_streams.at['PEVD-DROS','G']
        water_streams.loc['GPK-IND','G'] = water_streams.at['X-GPK','G']
        
        #Расчёт ГПК
        GPK = cotel.heatex('IND-GPK','GPK-out','X-GPK','GPK-IND',           KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
        gas_streams.loc['GPK-out','T':'G']   = [GPK[0],GPK[1],GPK[2],GPK[3]]
        water_streams.loc['GPK-IND','T':'G'] = [GPK[4],GPK[5],GPK[6],GPK[7]]
        
        #Баланс ППНД+ИНД+ГПК
        Qgas1 = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['GPK-out','H'])
        Qwat1 = water_streams.at['GPK-IND','G']*(water_streams.at['GPK-IND','H']-water_streams.at['X-GPK','H'])+water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['IND-PPND','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
        print('dQ/Q ППНД+ИНД+ГПК',(Qgas1-Qwat1)/Qgas1*100)
        if  abs((Qgas1-Qwat1)/Qgas1*100) < calctolerance:
            break;
    print("НД+ --- %s сек. ---" % round((time.time() - start_time),1))
    ######################################################################

    #Баланс общий
    Qgasall = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['GPK-out','H'])
    Qwatall = water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['X-GPK','H'])+water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['X-GPK','H'])-water_streams.at['BND-PEN','G']*(water_streams.at['PEN-EVD','H']-water_streams.at['BND-PEN','H'])
    print('dQ/Qsumm',(Qgasall-Qwatall)/Qgasall*100)
    if  abs((Qgasall-Qwatall)/Qgasall) < calctolerance:
        break;

print('Степень сухости пара в ЭВД: ',water.p_h(water_streams.at['EVD-IVD','P'],water_streams.at['EVD-IVD','H'])['Q'])
print('Степень сухости пара в ГПК: ',water.p_h(water_streams.at['GPK-IND','P'],water_streams.at['GPK-IND','H'])['Q'])
print("fin:--- %s сек. ---" % round((time.time() - start_time),1))

dQ/Q ПЕВД+ИВД 62.46991873466558
dQ/Q ПЕВД+ИВД -47.4315670703259
dQ/Q ПЕВД+ИВД 12.982833026675994
dQ/Q ПЕВД+ИВД -5.519994121270201
dQ/Q ПЕВД+ИВД 2.014364521973789
dQ/Q ПЕВД+ИВД -0.7818011920751982
dQ/Q ПЕВД+ИВД 0.2960278316769398
dQ/Q ПЕВД+ИВД -0.11307303295407485
dQ/Q ПЕВД+ИВД 0.04304708695953314
dQ/Q ПЕВД+ИВД -0.01640885368657202
dQ/Q ПЕВД+ИВД 0.006251774200359991
dQ/Q ПЕВД+ИВД -0.002382364226371088
dQ/Q ПЕВД+ИВД 0.0009077843128422202
dQ/Q ПЕВД+ИВД+ЭВД -0.6163840123570228
dQ/Q ПЕВД+ИВД -0.3404709923969816
dQ/Q ПЕВД+ИВД 0.12794068244235995
dQ/Q ПЕВД+ИВД -0.04825919403048551
dQ/Q ПЕВД+ИВД 0.018177447427394137
dQ/Q ПЕВД+ИВД -0.006850446426662782
dQ/Q ПЕВД+ИВД 0.002581172343409946
dQ/Q ПЕВД+ИВД -0.0009726311230497274
dQ/Q ПЕВД+ИВД+ЭВД -0.5269636259849011
dQ/Q ПЕВД+ИВД -0.29594902399654194
dQ/Q ПЕВД+ИВД 0.11019131446063885
dQ/Q ПЕВД+ИВД -0.04116214253377033
dQ/Q ПЕВД+ИВД 0.01535744301241749
dQ/Q ПЕВД+ИВД -0.005732414670262073
dQ/Q ПЕВД+ИВД 0.0021393533059804208
dQ/Q ПЕВД+ИВД -0.00079846341

In [2]:
#Вывод газовых потоков
gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
GTU-PEVD,542.100000,0.1,958.869199,251.9,0.7803,0.1237,0.0301,0.0594,0.0065
PEVD-IVD,462.986664,0.1,868.535732,251.9,0.7803,0.1237,0.0301,0.0594,0.0065
IVD-EVD,287.056702,0.1,673.565833,251.9,0.7803,0.1237,0.0301,0.0594,0.0065
EVD-PPND,222.792488,0.1,604.279946,251.9,0.7803,0.1237,0.0301,0.0594,0.0065
PPND-IND,220.296909,0.1,601.607981,251.9,0.7803,0.1237,0.0301,0.0594,0.0065
IND-GPK,170.375680,0.1,548.422890,251.9,0.7803,0.1237,0.0301,0.0594,0.0065
GPK-out,104.963028,0.1,479.397638,251.9,0.7803,0.1237,0.0301,0.0594,0.0065


In [3]:
#Вывод водяных потоков
water_streams

,T,P,H,G
PEVD-DROS,521.044504,8.657883,3444.041100,32.323652
IVD-PEVD,301.297556,8.746000,2747.104023,32.323652
EVD-IVD,281.379871,8.746000,1242.869097,32.323652
PEN-EVD,166.221575,8.746000,707.115426,32.323652
BND-PEN,164.738361,0.696400,696.095567,32.323652
PPND-TURB,209.373899,0.695379,2866.359658,6.417835
IND-PPND,164.738361,0.696400,2762.534980,6.417835
GPK-IND,164.731930,0.696400,696.067588,38.741487
X-GPK,60.000000,0.696400,251.748396,38.741487
SP1-OD,74.770000,0.038227,NaN,75.875000


In [4]:
#Невязка по всем параметрам
Qgas = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['PEVD-IVD','H'])
Qwat = water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['IVD-PEVD','H'])
print('dQ/Q ПЕВД+ИВД',((Qgas-Qwat)/Qgas*100))

Qgas1 = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['IVD-EVD','H'])
Qwat1 = water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['EVD-IVD','H'])
print('dQ/Q ПЕВД+ИВД+ЭВД',((Qgas1-Qwat1)/Qgas1*100))

Qgas = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['IND-GPK','H'])
Qwat = water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['IND-PPND','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
print('dQ/Q ППНД+ИНД',((Qgas-Qwat)/Qgas*100))

Qgas1 = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['GPK-out','H'])
Qwat1 = water_streams.at['GPK-IND','G']*(water_streams.at['GPK-IND','H']-water_streams.at['X-GPK','H'])+water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['IND-PPND','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
print('dQ/Q ППНД+ИНД+ГПК',(Qgas1-Qwat1)/Qgas1*100)

Qgasall = KPD*gas_streams.at['GTU-PEVD','G']*(gas_streams.at['GTU-PEVD','H']-gas_streams.at['GPK-out','H'])
Qwatall = water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['X-GPK','H'])+water_streams.at['PEVD-DROS','G']*(water_streams.at['PEVD-DROS','H']-water_streams.at['X-GPK','H'])-water_streams.at['BND-PEN','G']*(water_streams.at['PEN-EVD','H']-water_streams.at['BND-PEN','H'])
print('dQ/Qsumm',(Qgasall-Qwatall)/Qgasall*100)

dQ/Q ПЕВД+ИВД -0.00044935168648551217
dQ/Q ПЕВД+ИВД+ЭВД -0.0009539557612123557
dQ/Q ППНД+ИНД 0.0014830024824342982
dQ/Q ППНД+ИНД+ГПК 0.0006633137543019997
dQ/Qsumm -0.03295403602238618
